In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
import matplotlib.pyplot as plt
import joblib
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, r2_score
import joblib

# Custom wrapper for SARIMAX
class SARIMAXWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, order=(1, 0, 1), seasonal_order=(1, 0, 1, 24)):
        self.order = order
        self.seasonal_order = seasonal_order
    
    def fit(self, X, y):
        self.model = SARIMAX(y, exog=X, order=self.order, seasonal_order=self.seasonal_order)
        self.results = self.model.fit(disp=False)
        return self

    def predict(self, X):
        return self.results.get_forecast(steps=len(X), exog=X).predicted_mean
    
def mape_scorer(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100


# Load the dataset
df_raw = pd.read_csv('../../train.csv')

# Convert the 'date_time' column to datetime and sort the dataset
df_raw['date_time'] = pd.to_datetime(df_raw['date_time'])
df_raw.sort_values('date_time', inplace=True)

# Extracting non-numeric columns
non_numeric_cols = ['is_holiday', 'weather_type', 'weather_description']

# Group by 'date_time' and aggregate
agg_funcs = {col: 'mean' for col in df_raw.columns if col not in non_numeric_cols}
agg_funcs.update({col: lambda x: x.mode()[0] if not x.mode().empty else np.nan for col in non_numeric_cols})
df_aggregated = df_raw.groupby('date_time').agg(agg_funcs)

# One-hot encode categorical features
df = pd.get_dummies(df_aggregated, columns=non_numeric_cols, drop_first=True)

# Add hour from the index
df['hour'] = df.index.hour

# Split the dataset into features (X) and the target (y)
target = 'traffic_volume'
X = df.drop(target, axis=1)
y = df[target]

# Define a split index for a 90-10 train-test split
split_index = int(len(df) * 0.9)

# Split data
y_train = y.iloc[:split_index]
X_train = X.iloc[:split_index]
y_test = y.iloc[split_index:]
X_test = X.iloc[split_index:]

X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

# Adjusted parameter grid for Grid Search
param_grid = {
    'order': [(p, d, q) for p in range(1, 3) for d in range(1, 3) for q in range(1, 3)],
    'seasonal_order': [(p, d, q, 12) for p in range(1, 2) for d in range(1, 2) for q in range(1, 2)]
}

# Create the SARIMAX wrapper instance
sarimax = SARIMAXWrapper()

# Initialize GridSearchCV
grid_search = GridSearchCV(sarimax, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=1)

# Fit the GridSearchCV to find the best model
grid_search.fit(X_train, y_train)

# Use the best estimator to make predictions
best_sarimax = grid_search.best_estimator_
predicted_traffic_volume_sarimax = best_sarimax.predict(X_test)

# Evaluate metrics
def evaluate_metrics(y_true, y_pred):
    return {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mape_scorer(y_true, y_pred)
    }

train_metrics_sarimax = evaluate_metrics(y_train, best_sarimax.predict(X_train))
val_metrics_sarimax = evaluate_metrics(y_test, predicted_traffic_volume_sarimax)

# Print and compare metrics for SARIMAX
print("Training Set Metrics (SARIMAX):")
print(train_metrics_sarimax)

print("\nValidation Set Metrics (SARIMAX):")
print(val_metrics_sarimax)

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(y_train.index, y_train, label='Train')
plt.plot(y_test.index, y_test, label='Test')
plt.plot(y_test.index, predicted_traffic_volume_sarimax, label='SARIMAX Forecast', linestyle='--')
plt.legend()
plt.title('SARIMAX Forecast vs Actuals')
plt.show()

# Save the trained SARIMAX model
joblib.dump(best_sarimax, 'C_grid_sarimax_model.joblib')

C:\Users\Nadim Pramono\PycharmProjects\pythonProject1\v392\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Nadim Pramono\PycharmProjects\pythonProject1\v392\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Nadim Pramono\PycharmProjects\pythonProject1\v392\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Nadim Pramono\PycharmProjects\pythonProject1\v392\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Pred

In [ ]:
# Make predictions on the first 7 days of unseen data
forecast_unseen = best_sarimax.results.get_forecast(steps=7 * 24, exog=X_test.iloc[:7 * 24])
predicted_traffic_volume_unseen = forecast_unseen.predicted_mean

# Evaluate metrics on the unseen data for SARIMAX
unseen_metrics_sarimax = evaluate_metrics(y_test.iloc[:7 * 24], predicted_traffic_volume_unseen)

# Visualization for the first 7 days of unseen data
plt.figure(figsize=(12, 6))
plt.plot(y_test.index[:7 * 24], y_test.iloc[:7 * 24], label='Actual Unseen')
plt.plot(y_test.index[:7 * 24], predicted_traffic_volume_unseen, label='Forecast', linestyle='--')
plt.legend()
plt.title('Multivariate SARIMAX Forecast vs Actuals for the First 7 Days of Unseen Data')
plt.xlabel('Date')
plt.ylabel('Traffic Volume')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'date_time': y_test.index[:720],
    'actual_traffic_volume': y_test.iloc[:720].values,
    'forecasted_traffic_volume': predicted_traffic_volume_sarimax[:720]
})

# Display the DataFrame
print(comparison_df)

# Visualization
plt.figure(figsize=(15, 7))
plt.plot(comparison_df['date_time'], comparison_df['actual_traffic_volume'], label='Actual Traffic Volume', color='blue')
plt.plot(comparison_df['date_time'], comparison_df['forecasted_traffic_volume'], label='Forecasted Traffic Volume', color='orange', linestyle='--')
plt.xlabel('Date Time')
plt.ylabel('Traffic Volume')
plt.title('Actual vs Forecasted Traffic Volume (First 720 Hours)')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()